In [1]:
import os
from pyspark.sql import SparkSession

# Set GCS credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/longnguyen/credentials/key.json'

# Create Spark session with GCS connector
spark = SparkSession.builder \
    .appName("Test GCS with Spark") \
    .master("spark://node-2:7077") \
    .config("spark.jars.packages",
        "io.delta:delta-spark_2.13:4.0.0,"       
        "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.11"
    ) \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/longnguyen/credentials/key.json") \
    .getOrCreate()

# Verify connection
print(f"Spark Version: {spark.version}")
print(f"Master: {spark.sparkContext.master}")
print(f"App Name: {spark.sparkContext.appName}")
print("✓ Spark session with GCS connector created successfully!")

:: loading settings :: url = jar:file:/home/longnguyen/miniconda3/envs/spark_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/longnguyen/.ivy2.5.2/cache
The jars for the packages stored in: /home/longnguyen/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
com.google.cloud.bigdataoss#gcs-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6346ad31-4d2f-41c8-ac4e-f55b32f89de3;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	found com.google.cloud.bigdataoss#gcs-connector;hadoop3-2.2.11 in central
	found com.google.api-client#google-api-client-jackson2;2.0.1 in central
	found com.google.api-client#google-api-client;2.0.1 in central
	found com.google.oauth-client#google-oauth-client;1.34.1 in central
	found com.google.http-c

Spark Version: 4.0.0
Master: spark://node-2:7077
App Name: Test GCS with Spark
✓ Spark session with GCS connector created successfully!


In [2]:
# Write DataFrame to GCS bucket in different formats
# Replace 'your-bucket-name' with your actual GCS bucket name

GCS_BUCKET = "cdc-pipeline-data"  # Change this to your GCS bucket name
GCS_BASE_PATH = f"gs://{GCS_BUCKET}/uat/bronze"

In [3]:
# Verify the files were created by reading them back
print("=== Verifying files in GCS ===\n")


delta_path = f'{GCS_BASE_PATH}/products'
# Read Delta Lake
print("Reading Delta Lake from GCS:")
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()

print("✓ All files verified successfully!")

=== Verifying files in GCS ===

Reading Delta Lake from GCS:


25/12/25 14:56:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+---------+------+--------------------+--------------------+
|        json_payload|               topic|partition|offset|     kafka_timestamp| ingestion_timestamp|
+--------------------+--------------------+---------+------+--------------------+--------------------+
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     0|2025-12-25 13:02:...|2025-12-25 14:39:...|
|                NULL|cdc-pipeline.publ...|        0|     1|2025-12-25 13:02:...|2025-12-25 14:39:...|
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     2|2025-12-25 13:02:...|2025-12-25 14:39:...|
|                NULL|cdc-pipeline.publ...|        0|     3|2025-12-25 13:02:...|2025-12-25 14:39:...|
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     4|2025-12-25 13:02:...|2025-12-25 14:39:...|
|                NULL|cdc-pipeline.publ...|        0|     5|2025-12-25 13:02:...|2025-12-25 14:39:...|
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     6|2025-12-25 13

In [4]:
# Verify the files were created by reading them back
print("=== Verifying files in GCS ===\n")


delta_path = f'{GCS_BASE_PATH}/orders'
# Read Delta Lake
print("Reading Delta Lake from GCS:")
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()

print("✓ All files verified successfully!")

=== Verifying files in GCS ===

Reading Delta Lake from GCS:


+--------------------+--------------------+---------+------+--------------------+--------------------+
|        json_payload|               topic|partition|offset|     kafka_timestamp| ingestion_timestamp|
+--------------------+--------------------+---------+------+--------------------+--------------------+
|{"schema":{"type"...|cdc-pipeline.publ...|        2|     0|2025-12-25 13:03:...|2025-12-25 14:53:...|
|{"schema":{"type"...|cdc-pipeline.publ...|        2|     1|2025-12-25 13:03:...|2025-12-25 14:53:...|
|{"schema":{"type"...|cdc-pipeline.publ...|        1|     0|2025-12-25 13:03:...|2025-12-25 14:53:...|
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     0|2025-12-25 13:03:...|2025-12-25 14:53:...|
+--------------------+--------------------+---------+------+--------------------+--------------------+

✓ All files verified successfully!


In [4]:
df_delta.printSchema()

root
 |-- json_payload: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- kafka_timestamp: timestamp (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [ ]:
from pyspark.sql.functions import get_json_object

df_delta = df_delta.withColumn("product_id", get_json_object("json_payload", "$.payload.after.product_id"))
df_delta = df_delta.orderBy("kafka_timestamp", ascending=True)

df_delta.show()

+--------------------+--------------------+---------+------+--------------------+--------------------+----------+
|        json_payload|               topic|partition|offset|     kafka_timestamp| ingestion_timestamp|product_id|
+--------------------+--------------------+---------+------+--------------------+--------------------+----------+
|{"schema":{"type"...|cdc-pipeline.publ...|        1|     0|2025-12-22 16:49:...|2025-12-24 16:00:...|         3|
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     0|2025-12-22 16:49:...|2025-12-24 16:00:...|         4|
|{"schema":{"type"...|cdc-pipeline.publ...|        2|     0|2025-12-22 16:49:...|2025-12-24 16:00:...|         5|
|{"schema":{"type"...|cdc-pipeline.publ...|        2|     1|2025-12-22 16:49:...|2025-12-24 16:00:...|         6|
|{"schema":{"type"...|cdc-pipeline.publ...|        0|     1|2025-12-22 16:51:...|2025-12-24 16:00:...|         8|
|{"schema":{"type"...|cdc-pipeline.publ...|        2|     2|2025-12-25 11:20:...|2025-12

In [13]:
test_df = df_delta.filter(df_delta.product_id.isNull()).toPandas()

test_df

,json_payload,topic,partition,offset,kafka_timestamp,ingestion_timestamp,product_id
0,"{""schema"":{""type"":""struct"",""fields"":[{""type"":""...",cdc-pipeline.public.products,2,3,2025-12-25 11:45:04.164,2025-12-25 11:45:10.013,None
1,None,cdc-pipeline.public.products,2,4,2025-12-25 11:45:04.165,2025-12-25 11:45:10.013,None


In [16]:
print(test_df.loc[test_df['offset'] == 3, 'json_payload'].values[0])

{"schema":{"type":"struct","fields":[{"type":"struct","fields":[{"type":"int32","optional":false,"field":"product_id"},{"type":"string","optional":false,"field":"product_name"},{"type":"int32","optional":false,"field":"aisle_id"},{"type":"int32","optional":false,"field":"department_id"}],"optional":true,"name":"cdc-pipeline.public.products.Value","field":"before"},{"type":"struct","fields":[{"type":"int32","optional":false,"field":"product_id"},{"type":"string","optional":false,"field":"product_name"},{"type":"int32","optional":false,"field":"aisle_id"},{"type":"int32","optional":false,"field":"department_id"}],"optional":true,"name":"cdc-pipeline.public.products.Value","field":"after"},{"type":"struct","fields":[{"type":"string","optional":false,"field":"version"},{"type":"string","optional":false,"field":"connector"},{"type":"string","optional":false,"field":"name"},{"type":"int64","optional":false,"field":"ts_ms"},{"type":"string","optional":true,"name":"io.debezium.data.Enum","vers

In [17]:


# Stop Spark session
spark.stop()
print("✓ Spark session stopped")

✓ Spark session stopped
